<a href="https://colab.research.google.com/github/0xVolt/learn-langchain/blob/main/hf_models_locally_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using HuggingFace Models Locally with `LangChain`

## Ways to Create `LangChains` with HuggingFace Models

There are two wrappers that HuggingFace (HF) provides to use LLMs, one for a local run and host and another for hosting your models and projects on HuggingFaceHub (HFH). You'll see most projects using the latter approach. However, there are pros and cons for both.

## Why not `HuggingFaceHub`?

Plain and simple, the HFH doesn't support all models on HF. And, for the models that are supported, the wrappers only exist for the following tasks: `text2text-generation` and `text-generation`.

`text2text-generation` represents tasks that are accomplished by encoder-decoder models like BERT and BART.

`text-generation` represents tasks that are done by models that are solely decoders like GPT-2, etc.

It's important to understand what kind of model you're working with since the definition and parameters that you'd use to setup the LangChain will differ. Examples of this are to follow.

## About this Notebook

This notebook will compare setting up models with HFH and locally and then linking them to LangChain. The idea is to use this notebook for future reference if I need to make a project that involves fine-tuning any of these models to create a LangChain backed application.